# 1. Setup

## 1.1. Installing and importing libraries to the collab new session (therefore new environment)

In [1]:
!python -m nltk.downloader 'punkt'
#restart the runtime once installed

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install "tqdm==4.43.0"

In [3]:
!pip install sumy

     |████████████████████████████████| 83 kB 1.6 MB/s 
     |████████████████████████████████| 10.1 MB 32.4 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=da3798dea4405aa760755a0589cd948e222cdd78dc93371b3a9b367a5b62fc9a
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=a93a112c1b3477be624792a7c29406962550b32e8666a9577c571ad39c6f5974
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry


In [4]:
!pip install datasets

     |████████████████████████████████| 542 kB 5.2 MB/s 
     |████████████████████████████████| 118 kB 54.0 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 243 kB 39.0 MB/s 


In [5]:
!pip install rouge_score

In [6]:
# Importing necessary librairies
import datasets
from datasets import load_metric
import nltk
import numpy as np
import pandas as pd
import pickle 
from sklearn.model_selection import train_test_split
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.text_rank import TextRankSummarizer
import tqdm
from google.colab import drive

## 1.2. Importing personal drive to file in order to load the data


In [7]:
# Mount Google Drive to this Notebook instance.
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Data Formatting

## 2.1. Loading clean data 

In [8]:
# We load the cleaned data from a pickle format
file ='/content/drive/MyDrive/Colab Notebooks/Data/final_clean.pickle'
df = pd.read_pickle(file)
# Looking at the data
df

,description_narrative,TLDR,text_clean,text_embedding,summary_clean,summary_embedding,labels,labels_idx_list,doc_label
0,The training improved women’s knowledge on the...,The training improved women’s knowledge on the...,[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...",[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[0],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Illegal oil refining in the Niger Delta is inc...,CEHRD in an effort to create awareness on the ...,[Illegal oil refining in the Niger Delta is in...,"[[-0.26888534, -0.6092957, -0.23817927, -1.167...",[CEHRD in an effort to create awareness on the...,"[[-0.43648192, -0.071132354, -0.18081759, -0.4...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,CEHRD successfully set-up 4 formal and 4 infor...,CEHRD set-up the environmental clubs with the ...,[CEHRD successfully set-up 4 formal and 4 info...,"[[-0.48964322, -1.2085572, 1.0374501, 0.021369...",[CEHRD set-up the environmental clubs with the...,"[[-0.15335692, -0.29430717, 0.58692193, -1.126...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[10],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,PROJECT NAME : Delivering Accelerated Family P...,Delivering Accelerated Family Planning in Paki...,"[PROJECT NAME :, Delivering Accelerated Family...","[[-0.23698506, 0.15983863, -0.071198896, -1.20...",[Delivering Accelerated Family Planning in Pak...,"[[-0.50266284, -1.2923895, 0.4206834, -1.25673...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[1],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
4,Road traffic injuries are world's eighth leadi...,The Cardiff Trauma Pack Research and Develop...,[Road traffic injuries are world's eighth lead...,"[[-0.4666822, -1.19177, 0.9945381, -0.95759714...",[ The Cardiff Trauma Pack Research and Develo...,"[[-0.4679708, -0.43502182, 0.8388376, -1.47547...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[4, 5]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
...,...,...,...,...,...,...,...,...,...
2980,Metta has been responding to the humanitarian ...,"ProjectGoal: To improve the condition of 2,854...",[Metta has been responding to the humanitarian...,"[[-0.6875755, -1.0828758, 0.32236806, -1.33987...","[ProjectGoal:, To improve the condition of 2,8...","[[-0.58209735, 0.027911462, 0.29285246, -1.098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[7, 8]","[2980, 2980, 2980, 2980, 2980, 2980, 2980, 298..."
2981,"Destined Women is local not for profit, non-re...",GOAL: Contribute towards changing the socioeco...,"[Destined Women is local not for profit, non-r...","[[-0.3787009, -0.7928037, 0.16012278, -0.16681...","[GOAL:, Contribute towards changing the socioe...","[[-0.2746246, 0.04018628, 0.4832729, -0.996766...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1, 7]","[2981, 2981, 2981, 2981, 2981, 2981, 2981, 298..."
2982,The project will empower secondary school stud...,The project will empower secondary school stud...,[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...",[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2982, 2982, 2982, 2982, 2982, 2982, 2982, 2982]"
2983,The project has been specifically designed to ...,The project has been specifically designed to ...,[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...",[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2983, 2983, 2983, 2983, 2983, 2983]"


In [9]:
# Renaming the columns and
df = pd.DataFrame({'document': df['description_narrative'],
                   'summary': df['TLDR'],
                   'label' :  df['labels']
                   })

## 2.2. Splitting in training, validation and test datasets

In [10]:
# Using skicit learn library, we split the data, keeping 80% for training, 10% for validation and 10% for test data
# we note that in this notebook we will only use the test dataset as this is a baseline model 
train_dataset, test_dataset = train_test_split(df, test_size=0.2)
validation_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5)

# 3. Loading the summariser 

In [11]:
# Defining a function to parse, tokenize and summarise the text 
def summariser(doc):
  parser = PlaintextParser.from_string(doc, Tokenizer('english'))
  textrank = TextRankSummarizer()
  summary = textrank(parser.document, 2)
  return summary

# Applying the function to the dataset
#creating an empty list
text_rank_summaries = []

#looping in the texts of teh datasets
for doc in test_dataset['document']:
  #applying the function to each text 
  #appending the result to the list 
  text_rank_summaries.append(summariser(doc))

# Creating a column filled with the generated summaries 
test_dataset['predicted_summaries'] = text_rank_summaries
test_dataset = test_dataset.reset_index()
test_dataset

,index,document,summary,label,predicted_summaries
0,597,The development objective of the Balochistan I...,The development objective of the Balochistan I...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(The development objective of the Balochistan ...
1,1453,The development objective of the Accelerated F...,The development objective of the Accelerated F...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(The purpose of this extension request is to a...
2,1526,The overall objective is to ensure the equal p...,The overall objective is to ensure the equal p...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(The overall objective is to ensure the equal ...
3,537,The objective of the Saramacca Canal System (S...,The objective of the Saramacca Canal System (S...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(The objective of the Saramacca Canal System (...
4,1543,Sudan - Malaria - Federal Ministry of Health o...,This program has the following goals: Ensure a...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(This program has the following goals: Ensure ...
...,...,...,...,...,...
282,1452,The development objective of the Urban Water S...,The development objective of the Urban Water S...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(The development objective of the Urban Water ...
283,1279,nnTheProgramme to be supported by the Cities A...,TheProgramme to be supported by the Cities All...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(The cooperation shall encourage the ‘goodprac...
284,1966,The Tamil Nadu Irrigated Agriculture Moderniza...,The Tamil Nadu Irrigated Agriculture Moderniza...,"[1.0, 0.0, 0.0, 0.0, 0.0]",(The project objective is proposed to be achie...
285,1517,Ghana - HIV - West African Program to Combat A...,This program has the following goals: To reduc...,"[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",(This program has the following goals: To redu...


# 4. Evaluation

## 4.1. Data Preparation

In [12]:
# Creating lists of the reference and the predicted summaries
reference_summaries = test_dataset['summary']
predicted_summaries = test_dataset['predicted_summaries']

# Converting the predicted summaries to strings 
predicted_summaries = [str(sent) for sent in predicted_summaries]

# Defining a function to clean the predicted sentences from the added format generated by text_rank
def cleaner(sent):
  sent = sent.replace("(<Sentence: ", "")
  sent = sent.replace(" <Sentence:", "")
  sent = sent.replace ('>)', "")
  sent = sent.replace('>,', "")
  return sent 

# Applying the function to both the predicted and the reference summaries for consistency
predicted_summaries = [cleaner(sent) for sent in predicted_summaries]
reference_summaries = [cleaner(sent) for sent in reference_summaries]

## 4.2. Computing the ROUGE scores

In [13]:
# Loading the ROUGE metric from Hugginface 
metric = load_metric('rouge')
rouge_scores = metric.compute(predictions=predicted_summaries, references=reference_summaries)
rouge_scores

{'rouge1': AggregateScore(low=Score(precision=0.23462556224333309, recall=0.699742099053964, fmeasure=0.33810225896932183), mid=Score(precision=0.25409912305764276, recall=0.7330251882640637, fmeasure=0.36231674344303055), high=Score(precision=0.274423584000407, recall=0.7643098935470596, fmeasure=0.3865014014500577)),
 'rouge2': AggregateScore(low=Score(precision=0.17064905167923866, recall=0.48778519361022316, fmeasure=0.2426499865741622), mid=Score(precision=0.19154541362950772, recall=0.5346237545782319, fmeasure=0.27041801816190736), high=Score(precision=0.21349168700432616, recall=0.5843378064093773, fmeasure=0.29854401760601085)),
 'rougeL': AggregateScore(low=Score(precision=0.20685487759906654, recall=0.6052353123894064, fmeasure=0.2962171052937976), mid=Score(precision=0.22688409335133852, recall=0.6461505966924616, fmeasure=0.3214144475555307), high=Score(precision=0.246680015508932, recall=0.6801903352064047, fmeasure=0.3463721630148046)),
 'rougeLsum': AggregateScore(low=S

## 4.3. View a few examples

In [20]:
print('This is the reference summary: ')
print(reference_summaries[16])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[16])

This is the reference summary: 
The objective of the Solid Waste Management Program Project for Colombia is to support the implementation of the solid waste management component of the Borrower's program which aims at improving the quality and coverage of integrated solid waste management services in the territory of the Borrower. 
   
This is the predicted summary: 
The objective of the Solid Waste Management Program Project for Colombia is to support the implementation of the solid waste management component of the Borrower's program which aims at improving the quality and coverage of integrated solid waste management services in the territory of the Borrower. The component will primarily finance the construction of solid waste management infrastructure consisting of civil works, equipment and services in support of the rehabilitation, construction, and/or expansion of landfills and transfer stations in selected sites and the closing of non sanitary open-air landfills.


In [21]:
print('This is the reference summary: ')
print(reference_summaries[69])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[69])

This is the reference summary: 
The projects objectives therefore are to:n	Contribute to greater awareness among companies in the South and emergingeconomies of their responsibility to respect under the UNGPs;n	Build the capacity of companies and civil society actors in those countries to support improved business respect for human rights on the ground; andn	Share the learning gained among a wider audienceof interested stakeholders through a new, independent online portal.
   
This is the predicted summary: 
Only a relatively small number of companies in the Global South and emerging economies are actively engaged in the topic of business and human rights, and an even smaller group have publicly available policies or information about their human rights due diligenceapproaches. There is an urgent need to scale up learning about what works in meeting the responsibility to respect in the Global South and emerging economies, including among small and medium-size companies (SMEs), and 

In [22]:
print('This is the reference summary: ')
print(reference_summaries[103])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[103])

This is the reference summary: 
This project promotes the creation of inclusive political spaces for LGBTI+ persons in ASEAN by using cultural strategies for advocacy in order to advance alternative and people-centered narratives regionalism in Southeast Asia. 
   
This is the predicted summary: 
These cultural materials are aimed to become a starting point in forming an LGBTI+ collective memory as well as innovative instruments for advocacy.nThe project entails 4 key activities: Organize discussions on LGBTI+ perspectives on alternative regionalism and regional solidarity; produce historical-cultural narratives inclusive of LGBTI+ persons in Southeast Asia to engage ASEAN officials; and organize a Queer Festival as part of the annual ASEAN Civil Society Conference in 2020. Such cultural instruments help to counter extremist propaganda which justifies the marginalisation and persecution of people based on sexual orientation, gender identity and/or sexual characteristics (SOGIESC), and 